In [46]:
# libraries
import pandas as pd
import numpy as np
from konlpy.tag import Okt
from konlpy.tag import Kkma
okt = Okt()
kkma = Kkma()
from collections import Counter
import pickle
import os
import re
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from transformers import T5Tokenizer, T5ForConditionalGeneration
import nltk
nltk.download('punkt')

# !pip install transformers[torch]==4.27.4

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Andlab\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# load
review_df = pd.read_csv('C:/Users/Andlab/jupyter_study/TextminingProject/concatdataset/review_df.csv')
print(len(review_df))

48470


#### 전처리

In [ ]:
# check
review_df

,직무,고용 현황,근무지,날짜,별점,요약,장점,단점,경영진에게 바라는 점
0,금융/재무,현직원,서울,NaN,5.0,"""수평적문화 , 워라밸 ,복지포인트 많이줌, 직원의견 긍정적으로 수렴.""",\n자동차보험 시장점유율이 빠르게 확보되는 중으로 성장성이 기대됨. 대물직원은 자회...,"\n일이 많고, 체계가 좀 부족하다는 점.차차 나아질거라고 봅니다.\n",\n앞으로 더 적극적으로 마케팅하고 내실을 다져 탄탄한 보험회사로 성장하길 기원합니...
1,디자인,현직원,서울,NaN,4.0,"""스타트업의 성격이 있어, 개인의 역할이 중요하다! 운영부터 구축까지 다양한 범위의...","\n연차를 개인의 스케줄에 맞게 사용할 수 있다. 부서마다 다르겠지만, 다양성을 존...",\n기본으로 제공되는 물품이 너무 적다.. 커피머신은 있지만 캡슐은 개인 구비해야하...,\n아직은 체계가 덜 잡혀 있는 느낌이 있습니다. 앞으로 더 체계가 잡혔으면 좋겠습...
2,금융/재무,현직원,서울,NaN,5.0,"""워라밸 균형, 시차 출근제, 수평적인 분위기, 영어이름, 젊은 사람들이 많음""",\n직급 또는 직책으로 부르지 않고 스스로 지은 영어이름으로 부르며 사무실 분위기는...,"\n부서간 알앤알이 명확하지않고, 명확하지 않은 만큼 일을 미루는 경향이 있기도 합...",\n작은 회사인만큼 직원들의 말에 귀기울여주기 바랍니다.\n
3,기획/경영,전직원,서울,NaN,5.0,"""하고자하는 의욕이 있다면 충분히 성장하고 발전할 수 있는 곳""",\n연차를 마음대로 씀. 기업이 커나가는 과정을 직접 눈으로 지켜볼 수 있음. 기획...,\n주니어가 많아서 일을 차근차근 배운다기 보다 부딪치면서 배운다\n,\n경영진의 생각이 직원들에게 잘 전달될 수 있었으면 좋겠다\n
4,금융/재무,전직원,서울,2023. 08,3.0,"""팀바팀이지만 서로 단합이 안되고 깎아내리기가 난무한 기업. 중재자도 없고 서로 일...",\n워라빌이 철저히 지켜짐. 팀 사람들은 좋음. 전체적으로 활기찬 느낌\n,\n불필요한 회의와 에너지 낭비가 많음. 그 에너지를 일하는대 쏟으면 좋갰음\n,\n제발 직원들의 목소리에 귀를 기울여주었으면. 쓸데없는 보고와 회의 좀 줄이세요.\n
...,...,...,...,...,...,...,...,...,...
48465,영업/제휴,전직원,경기,2019. 06,3.0,"""위치는 좀 멀지만 회사 시설이 좋고 외관이 예쁘다. 구내식당이 있어 식사해결 가능...",\n구내식당이 있어 식비가 안들어서 좋다. 시설이 깨끗하고 화장실이 깔끔해서 이용하...,\n사무실 청소해주는 사람이 없어서 매우 더럽다. 임원방은 막내들이 청소함. 야근이...,\n건물 외관과 환경만 신경쓰지 말고 직원복지에 좀 더 신경썼으면. 사무실 내도 청...
48466,전문직,전직원,해외,2019. 06,1.0,"""회사가 30년이 이상 운영되어서 그런지...\r좀 고였습니다.""",\n예전에는 돈이라고 했던 것 같은데.... 저는 별로 와 닿지 않네요...\r영업...,\n회사가 속이 빈 느낌 입니다.\r안을 채우고 싶어도 기존에 계신 분들이 좀 갑갑...,\n바이어도 중요하지만 종업원도 내부고객이라는 점 \r생각해주시면 고맙습니다.\n
48467,영업/제휴,현직원,해외,2019. 05,3.0,"""약간 올드하지만 새롭게 가려는 경향이 있음. 근데 따라가기가 쉽지 않음.""","\n연봉이 동종 업계에서는 많은편에 속하고, 업무강도가 빡쎄기에 다른데 이직하면 좀...","\n같은 보고서가 여러개, 시스템을 사용하는데 그걸 사용하지 않고 계속 취합함.\n",\n휴가 관련에 좀 도 신경이 필요합니다\n
48468,생산/제조,전직원,경기,2019. 04,2.0,"""연봉은 괜찮았지만 나머지는 다 별로였던 회사..바이어 앞에 납작 엎드려야 하고 팀...",\n연봉은 그냥 평범.인센을 잘줌.사무실 외관이 이쁘고 핸드백 제조 업계에서는 넘버...,\n연차 쓰는거에 눈치 엄청 줌. 고인물 많고 직원 복지 없었던걸로 기억... 동네...,\n직원들을 생각해 주세요. 매출만 잘나온다고 좋은 회사는 아니라고 생각합니다. 제...


In [ ]:
# 직무 unique
review_df['직무'].unique()

array(['금융/재무', '디자인', '기획/경영', '개발', '마케팅/시장조사', '전문직', '서비스/고객지원',
       '데이터', '인사/총무', '교육', '영업/제휴', '생산/제조', '미디어/홍보', '유통/무역', '기타',
       '엔지니어링', nan, '연구개발', '생산관리/품질관리', '특수계층/공공',
       '\n          현직원\n      ', '\n          전직원\n      ', '법률/법무',
       '의약'], dtype=object)

In [ ]:
# 결측치 데이터 제거
review_df = review_df[(review_df['직무']!='\n          전직원\n      ')&(review_df['직무']!='\n          현직원\n      ')&(review_df['직무'].notna())]
len(review_df)

47827

In [ ]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47827 entries, 0 to 48469
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   직무           47827 non-null  object 
 1   고용 현황        47827 non-null  object 
 2   근무지          47827 non-null  object 
 3   날짜           47266 non-null  object 
 4   별점           47827 non-null  float64
 5   요약           47827 non-null  object 
 6   장점           47827 non-null  object 
 7   단점           47827 non-null  object 
 8   경영진에게 바라는 점  47827 non-null  object 
dtypes: float64(1), object(8)
memory usage: 3.6+ MB


In [ ]:
# 장점
import re

review_df['장점'] = review_df['장점'].str.replace("\n"," ")
review_df['장점'] = review_df['장점'].str.replace("\r"," ")
review_df['장점'] = review_df['장점'].str.replace("\t"," ")
review_df['장점'] = review_df['장점'].str.replace(",","")
# review_df['장점'] = review_df['장점'].str.replace(".","")

In [ ]:
# 단점
review_df['단점'] = review_df['단점'].str.replace("\n"," ")
review_df['단점'] = review_df['단점'].str.replace("\r"," ")
review_df['단점'] = review_df['단점'].str.replace("\t"," ")
review_df['단점'] = review_df['단점'].str.replace(",","")
# review_df['단점'] = review_df['단점'].str.replace(".","")

In [ ]:
# 요약
review_df['요약'] = review_df['요약'].str.replace("\n"," ")
review_df['요약'] = review_df['요약'].str.replace("\r"," ")
review_df['요약'] = review_df['요약'].str.replace("\t"," ")
review_df['요약'] = review_df['요약'].str.replace(",","")
# review_df['요약'] = review_df['요약'].str.replace(".","")

In [ ]:
# 정규표현식
review_df['장점']= review_df['장점'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)
review_df['단점']= review_df['단점'].str.replace(pat=r'[^\w]',repl=r' ',regex=True)

In [ ]:
review_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 47827 entries, 0 to 48469
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   직무           47827 non-null  object 
 1   고용 현황        47827 non-null  object 
 2   근무지          47827 non-null  object 
 3   날짜           47266 non-null  object 
 4   별점           47827 non-null  float64
 5   요약           47827 non-null  object 
 6   장점           47827 non-null  object 
 7   단점           47827 non-null  object 
 8   경영진에게 바라는 점  47827 non-null  object 
dtypes: float64(1), object(8)
memory usage: 3.6+ MB


In [ ]:
# # okt 정규화
# for i, review in enumerate(review_df['장점']):
#     review_df.loc[i,'장점'] =  okt.normalize(review)

# for i, review in enumerate(review_df['단점']):
#     review_df.loc[i,'단점'] =  okt.normalize(review)

In [ ]:
# 장단점 column 생성
review_df['장단점'] = review_df['장점']+review_df['단점']

C:\Users\Andlab\AppData\Local\Temp\ipykernel_2476\1247148360.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  review_df['장단점'] = review_df['장점']+review_df['단점']


In [ ]:
# 저장
# review_df.to_csv('C:/Users/Andlab/jupyter_study/TextminingProject/review_summarization/review_df.csv',encoding='utf-8',index=False)

In [ ]:
# 저장된 파일 불러오기
review_df = pd.read_csv('C:/Users/Andlab/jupyter_study/TextminingProject/review_summarization/review_df.csv')
review_df[:3]

,직무,고용 현황,근무지,날짜,별점,요약,장점,단점,경영진에게 바라는 점,장단점
0,금융/재무,현직원,서울,NaN,5.0,"""수평적문화 워라밸 복지포인트 많이줌 직원의견 긍정적으로 수렴.""",자동차보험 시장점유율이 빠르게 확보되는 중으로 성장성이 기대됨 대물직원은 자회사...,일이 많고 체계가 좀 부족하다는 점 차차 나아질거라고 봅니다,\n앞으로 더 적극적으로 마케팅하고 내실을 다져 탄탄한 보험회사로 성장하길 기원합니...,자동차보험 시장점유율이 빠르게 확보되는 중으로 성장성이 기대됨 대물직원은 자회사...
1,디자인,현직원,서울,NaN,4.0,"""스타트업의 성격이 있어 개인의 역할이 중요하다! 운영부터 구축까지 다양한 범위의 ...",연차를 개인의 스케줄에 맞게 사용할 수 있다 부서마다 다르겠지만 다양성을 존중하...,기본으로 제공되는 물품이 너무 적다 커피머신은 있지만 캡슐은 개인 구비해야하고...,\n아직은 체계가 덜 잡혀 있는 느낌이 있습니다. 앞으로 더 체계가 잡혔으면 좋겠습...,연차를 개인의 스케줄에 맞게 사용할 수 있다 부서마다 다르겠지만 다양성을 존중하...
2,금융/재무,현직원,서울,NaN,5.0,"""워라밸 균형 시차 출근제 수평적인 분위기 영어이름 젊은 사람들이 많음""",직급 또는 직책으로 부르지 않고 스스로 지은 영어이름으로 부르며 사무실 분위기는 ...,부서간 알앤알이 명확하지않고 명확하지 않은 만큼 일을 미루는 경향이 있기도 합니다...,\n작은 회사인만큼 직원들의 말에 귀기울여주기 바랍니다.\n,직급 또는 직책으로 부르지 않고 스스로 지은 영어이름으로 부르며 사무실 분위기는 ...


#### 테마 키워드 추출

In [ ]:
# 형태소 종류 확인
okt.tagset

{'Adjective': '형용사',
 'Adverb': '부사',
 'Alpha': '알파벳',
 'Conjunction': '접속사',
 'Determiner': '관형사',
 'Eomi': '어미',
 'Exclamation': '감탄사',
 'Foreign': '외국어, 한자 및 기타기호',
 'Hashtag': '트위터 해쉬태그',
 'Josa': '조사',
 'KoreanParticle': '(ex: ㅋㅋ)',
 'Noun': '명사',
 'Number': '숫자',
 'PreEomi': '선어말어미',
 'Punctuation': '구두점',
 'ScreenName': '트위터 아이디',
 'Suffix': '접미사',
 'Unknown': '미등록어',
 'Verb': '동사'}

In [ ]:
# 장단점 모든 데이터에서 테마 키워드 추출
words_noun = [] # 형태소가 명사인 경우 담을 키워드 리스트 생성

for review in review_df['장단점']:
    pos = okt.nouns(review) # 명사만 추출
    words_noun.extend(pos)
# print(words_noun)

In [ ]:
# element별 개수 세기
count_noun = Counter(words_noun)

In [ ]:
# pickle로 저장 & 읽어오기

# pickle로 저장
# with open('count_noun.pkl', 'wb') as tf:
# 	pickle.dump(count_noun, tf)

# pickle 읽어오기
with open('count_noun.pkl', 'rb')as tf:
	count_noun = pickle.load(tf)

In [ ]:
# count수가 많은 상위 100개
dict({key: value for key, value in sorted(count_noun.items(), key = lambda item: item[1], reverse=True)[:100]})

##### 테마 분류
- 워라벨: 업무, 일, 야근, 근무, 시간, 퇴근, 자유, 경험, 성장, 능력, 기회, 강도
- 회사 분위기: 직원, 분위기, 팀, 눈치, 문화, 체계, 수평, 환경, 조직, 제도
- 복지: 복지, 연차, 휴가, 식대
- 연봉: 연봉, 급여, 월급, 돈
- 회사 위치: 출퇴근, 위치 (개인마다 차이 있을 수도..)

### 테마별 문장 나누기

#### 직무별 긍정/부정별 데이터 나누기

In [ ]:
# check
split_review = pd.read_csv('C:/Users/Andlab/jupyter_study/TextminingProject/concatdataset/review_df.csv')

### 전처리 #########
# 결측치 데이터 제거
split_review = split_review[(split_review['직무']!='\n          전직원\n      ')&(split_review['직무']!='\n          현직원\n      ')&(split_review['직무'].notna())]

# replace
split_review['장점'] = split_review['장점'].str.replace(",","")
split_review['장점'] = split_review['장점'].str.replace("\r","")
split_review['장점'] = split_review['장점'].str.replace("\n","")
split_review['단점'] = split_review['단점'].str.replace(",","")
split_review['단점'] = split_review['단점'].str.replace("\r","")
split_review['단점'] = split_review['단점'].str.replace("\n","")


split_review.head()

,직무,고용 현황,근무지,날짜,별점,요약,장점,단점,경영진에게 바라는 점
0,금융/재무,현직원,서울,NaN,5.0,"""수평적문화 , 워라밸 ,복지포인트 많이줌, 직원의견 긍정적으로 수렴.""",자동차보험 시장점유율이 빠르게 확보되는 중으로 성장성이 기대됨. 대물직원은 자회사 ...,일이 많고 체계가 좀 부족하다는 점.차차 나아질거라고 봅니다.,\n앞으로 더 적극적으로 마케팅하고 내실을 다져 탄탄한 보험회사로 성장하길 기원합니...
1,디자인,현직원,서울,NaN,4.0,"""스타트업의 성격이 있어, 개인의 역할이 중요하다! 운영부터 구축까지 다양한 범위의...",연차를 개인의 스케줄에 맞게 사용할 수 있다. 부서마다 다르겠지만 다양성을 존중하고...,기본으로 제공되는 물품이 너무 적다.. 커피머신은 있지만 캡슐은 개인 구비해야하고 ...,\n아직은 체계가 덜 잡혀 있는 느낌이 있습니다. 앞으로 더 체계가 잡혔으면 좋겠습...
2,금융/재무,현직원,서울,NaN,5.0,"""워라밸 균형, 시차 출근제, 수평적인 분위기, 영어이름, 젊은 사람들이 많음""",직급 또는 직책으로 부르지 않고 스스로 지은 영어이름으로 부르며 사무실 분위기는 수...,부서간 알앤알이 명확하지않고 명확하지 않은 만큼 일을 미루는 경향이 있기도 합니다....,\n작은 회사인만큼 직원들의 말에 귀기울여주기 바랍니다.\n
3,기획/경영,전직원,서울,NaN,5.0,"""하고자하는 의욕이 있다면 충분히 성장하고 발전할 수 있는 곳""",연차를 마음대로 씀. 기업이 커나가는 과정을 직접 눈으로 지켜볼 수 있음. 기획부터...,주니어가 많아서 일을 차근차근 배운다기 보다 부딪치면서 배운다,\n경영진의 생각이 직원들에게 잘 전달될 수 있었으면 좋겠다\n
4,금융/재무,전직원,서울,2023. 08,3.0,"""팀바팀이지만 서로 단합이 안되고 깎아내리기가 난무한 기업. 중재자도 없고 서로 일...",워라빌이 철저히 지켜짐. 팀 사람들은 좋음. 전체적으로 활기찬 느낌,불필요한 회의와 에너지 낭비가 많음. 그 에너지를 일하는대 쏟으면 좋갰음,\n제발 직원들의 목소리에 귀를 기울여주었으면. 쓸데없는 보고와 회의 좀 줄이세요.\n


In [ ]:
# # 문장 나누기: 성능 안 좋음
# # kiwi
# from kiwipiepy import Kiwi
# kiwi = Kiwi()
# kiwi.split_into_sents(review_df['장단점'][4])

# # kkma
# kkma.sentences(review_df['장단점'][4])

[Sentence(text='워라빌이 철저히 지켜짐  팀 사람들은 좋음  전체적으로 활기찬 느낌  불필요한 회의와 에너지 낭비가 많음', start=1, end=59, tokens=None, subs=[]),
 Sentence(text='그 에너지를 일하는대 쏟으면 좋갰음', start=61, end=80, tokens=None, subs=[])]

추가) ~~'음'으로 끝나는 부분에서 split

In [ ]:
def makedirs(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
# 폴더 생성
makedirs('./review_sentences_by_job')

In [ ]:
job = ['금융/재무', '디자인', '기획/경영', '개발', '마케팅/시장조사', '전문직', '서비스/고객지원',
       '데이터', '인사/총무', '교육', '영업/제휴', '생산/제조', '미디어/홍보', '유통/무역', '기타',
       '엔지니어링', '연구개발', '생산관리/품질관리', '특수계층/공공', '법률/법무', '의약']

for i in range(len(job)):
    job_name = job[i]
    job_pos = []
    job_neg = []
    job_pos_sen = []
    job_neg_sen = []
    job_df = split_review[split_review['직무']==job[i]]

    # positive ###
    job_pos.extend(job_df['장점'])
    for sentence in job_pos:
        if re.search(r'[0-9].', sentence):
            job_pos_sen.extend(re.split(r'[0-9].', sentence))
        elif '.' in sentence:
            job_pos_sen.extend(sentence.split('.'))
        elif '/' in sentence:
            job_pos_sen.extend(sentence.split('/'))
        else:
            job_pos_sen.append(sentence)
    # pickle로 저장
    with open('./review_sentences_by_job/{}_pos.pkl'.format(job_name.replace('/','')), 'wb') as tf:
        pickle.dump(job_pos_sen, tf)

    # negative ###
    job_neg.extend(job_df['단점'])
    for sentence in job_neg:
        if re.search(r'[0-9].', sentence):
            job_neg_sen.extend(re.split(r'[0-9].', sentence))
        elif '.' in sentence:
            job_neg_sen.extend(sentence.split('.'))
        elif '/' in sentence:
            job_neg_sen.extend(sentence.split('/'))
        else:
            job_neg_sen.append(sentence)
    # pickle로 저장
    with open('./review_sentences_by_job/{}_neg.pkl'.format(job_name.replace('/','')), 'wb') as tf:
        pickle.dump(job_neg_sen, tf)

#### 직무 안에서 테마별 데이터 나누기

In [ ]:
# 직무별 - 테마별 데이터 저장할 폴더 생성
makedirs('./review_by_job_theme')

In [ ]:
# theme별 키워드 사전
balance = ['업무', '야근', '근무', '시간', '퇴근', '자유', '경험', '성장', '능력', '기회', '강도']
environment = ['직원', '분위기', '팀', '눈치', '문화', '체계', '수평', '환경', '조직', '제도']
benefit = ['복지', '연차', '휴가', '식대','자율출퇴근']
salary = ['연봉', '급여', '월급', '돈']

In [ ]:
## 폴더 안에 있는 pkl 파일 리스트 가져오기
path = './review_sentences_by_job/'
file_list = os.listdir(path)
file_list_name = [file for file in file_list if file.endswith('.pkl')]

# 파일 하나씩 읽어오기
for file in file_list_name:
    filename = file.split('.pkl')[0]

    with open('./review_sentences_by_job/{}'.format(file), 'rb')as tf:
	    sentence_list = pickle.load(tf)

    # '' 빈 값 제거
    sentence_list = [v for v in sentence_list if v]

    # 테마별 문장 담을 리스트 생성 (첫번째 값 '' 부여)
    balance_sen = ['']
    environment_sen = ['']
    benefit_sen = ['']
    salary_sen = ['']

    for sentence in sentence_list:
        # balance
        for bal in balance: # 사전
            if bal in sentence:
                if sentence != balance_sen[-1]:
                    balance_sen.append(sentence)
                else:
                    continue
        
        # environment
        for env in environment: # 사전
            if env in sentence:
                if sentence != environment_sen[-1]:
                    environment_sen.append(sentence)
                else:
                    continue
        
        # benefit
        for ben in benefit: # 사전
            if ben in sentence:
                if sentence != benefit_sen[-1]:
                    benefit_sen.append(sentence)
                else:
                    continue
        
        # salary
        for sal in salary: # 사전
            if sal in sentence:
                if sentence != salary_sen[-1]:
                    salary_sen.append(sentence)
                else:
                    continue
    
    # 첫번째 값 '' 삭제
    balance_sen.pop(0)
    environment_sen.pop(0)
    benefit_sen.pop(0)
    salary_sen.pop(0)

    # pickle로 저장
    with open('./review_by_job_theme/{}_balance.pkl'.format(filename), 'wb') as tf:
        pickle.dump(balance_sen, tf)
    
    with open('./review_by_job_theme/{}_environment.pkl'.format(filename), 'wb') as tf:
        pickle.dump(environment_sen, tf)
    
    with open('./review_by_job_theme/{}_benefit.pkl'.format(filename), 'wb') as tf:
        pickle.dump(benefit_sen, tf)
    
    with open('./review_by_job_theme/{}_salary.pkl'.format(filename), 'wb') as tf:
        pickle.dump(salary_sen, tf)


### 직무별-테마별 요약

* https://github.com/AIRC-KETI/ke-t5
* https://github.com/AIRC-KETI/ke-t5#pretrained-models
* https://github.com/AIRC-KETI/ke-t5-downstreams
* https://huggingface.co/docs/transformers/model_doc/t5

In [13]:
# pickle 읽어오기
with open('./review_by_job_theme/개발_neg_salary.pkl', 'rb')as tf:
	data_test = pickle.load(tf)
data_test[:3]

['직급 체계가 없음 업무 프로세스도 비효율 본부 별 협업 및 의사소통 비원활 낮은 연봉',
 '금융권 치고 낮은 연봉부서마다 분위기 너무 다름 업무강도 복불복인 것 같음',
 '연봉계약 이외에 하나도 없는 복지 그냥 없다고 보면 됨']

In [38]:
# 데이터 하나의 요소로 합치기
sentence_all = ' '.join(data_test)

In [67]:
max_input_length = 128

In [85]:
model_dir = "KETI-AIR/ke-t5-base-ko"
tokenizer = T5Tokenizer.from_pretrained(model_dir)
model = T5ForConditionalGeneration.from_pretrained(model_dir)

spiece.model:   0%|          | 0.00/1.47M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.96k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/602 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

In [86]:
def summary(text):

    # input_ids = tokenizer.encode(text)

    inputs = ["summarize: " + text]

    inputs = tokenizer(inputs, max_length=max_input_length, truncation=True, return_tensors="pt")

    summary_ids = model.generate(
                                # torch.tensor([input_ids]),
                                **inputs,
                                do_sample = True,
                                num_beams = 8,
                                min_length = 10,
                                max_length = 32
                                )
    # decoded_output = tokenizer.decode(summary_ids.squeeze().tolist(), skip_special_tokens = True)
    decoded_output = tokenizer.batch_decode(summary_ids, skip_special_tokens=True)[0]
    predicted_title = nltk.sent_tokenize(decoded_output.strip()) #[0]
    # predicted = nltk.sent_tokenize(decoded_output.strip())[0]
    return predicted_title

In [87]:
summary(sentence_all)

['OVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVEROVER']

In [91]:
# pip install tensorflow
# pip install tensorflow tensorflow_text
import numpy as np
import tensorflow as tf
# import tensorflow_text

In [108]:
model_path = "C:\\Users\\Andlab\\jupyter_study\\TextminingProject\\review_summarization\\t5_small"
loaded = tf.saved_model.load(model_path)
infer = loaded.signatures["serving_default"]

input_str = sentence_all

input_str_topic = "summarize_topic: " + input_str
input_str_summary = "summarize_summary: " + input_str

x = tf.constant([input_str_topic])

result = infer(x)
print([out.decode('utf-8') for out in result['inputs'].numpy()])
print([out.decode('utf-8') for out in result['outputs'].numpy()])

OSError: SavedModel file does not exist at: C:\Users\Andlab\jupyter_study\TextminingProject\review_summarization\t5_small\{saved_model.pbtxt|saved_model.pb}

데이터, 디자인